In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [40]:
# Cargar el dataset limpio sin modificarlo
file_path = '/Users/chants/Desktop/Pset2_cucalon/data/processed/Airbnb_Cleaned.csv'
df_cleaned = pd.read_csv(file_path)

# Crear una copia para trabajar en Feature Engineering sin afectar el dataset original
df = df_cleaned.copy()

print("✅ Copia del dataset creada exitosamente. Listo para Feature Engineering")

✅ Copia del dataset creada exitosamente. Listo para Feature Engineering


In [ ]:
import re  #  Importamos regex para extraer valores correctamente

# Crear copia del dataset para no modificar el original
df_featured = df.copy()

# Función para contar el número de amenidades
def count_amenities(amenities_str):
    if isinstance(amenities_str, str):  
        # Expresión regular para capturar palabras dentro de comillas dobles ""
        matches = re.findall(r'"([^"]+)"', amenities_str)  
        return len(matches)  #  Retorna la cantidad de amenidades detectadas
    return 0  # Si no es string, devuelve 0

# Aplicar la función a la columna 'amenities'
df_featured['total_amenities'] = df_featured['amenities'].apply(count_amenities)

# Verificar los primeros valores
print(df_featured[['amenities', 'total_amenities']].head(10))

# Correlación con `log_price`
correlation = df_featured[['total_amenities', 'log_price']].corr()
print("\n📈 Correlación de total_amenities con log_price:")
print(correlation)

# Guardar las columnas en un archivo CSV
output_path = "/Users/chants/Desktop/Pset2_cucalon/data/processed/amenities_feature.csv"
df_featured[['amenities', 'total_amenities']].to_csv(output_path, index=False)


                                           amenities  total_amenities
0  {"Wireless Internet","Air conditioning",Kitche...                5
1  {"Wireless Internet","Air conditioning",Kitche...                7
2  {TV,"Cable TV","Wireless Internet","Air condit...               11
3  {TV,"Cable TV",Internet,"Wireless Internet",Ki...                8
4  {TV,Internet,"Wireless Internet","Air conditio...                6
5  {TV,"Wireless Internet",Heating,"Smoke detecto...                6
6  {TV,Internet,"Wireless Internet","Air conditio...               10
7  {TV,"Cable TV","Wireless Internet","Wheelchair...               14
8  {TV,"Cable TV","Wireless Internet","Pets live ...               14
9  {"Wireless Internet","Air conditioning",Kitche...                7

📈 Correlación de total_amenities con log_price:
                 total_amenities  log_price
total_amenities         1.000000   0.141215
log_price               0.141215   1.000000


In [ ]:

# Calcular el precio promedio en un radio de 1KM (aproximado a 0.01 en coordenadas)
def calculate_avg_price(lat, lon, df, radius=0.01):
    nearby = df[(abs(df['latitude'] - lat) < radius) & (abs(df['longitude'] - lon) < radius)]
    return nearby['log_price'].mean() if not nearby.empty else df['log_price'].mean()

df_featured['avg_price_1km'] = df_featured.apply(lambda row: calculate_avg_price(row['latitude'], row['longitude'], df_featured), axis=1)

# Correlación con log_price
correlation = df_featured[['avg_price_1km', 'log_price']].corr()

# Mostrar los primeros valores de la nueva variable
print(df_featured[['latitude', 'longitude', 'avg_price_1km']].head(10))

# Mostrar la correlación
print("\n📈 Correlación de avg_price_1km con log_price:")
print(correlation)

# 📌 Guardar las columnas en un archivo CSV
output_path = "/Users/chants/Desktop/Pset2_cucalon/data/processed/ubi_feature.csv"
df_featured[['latitude', 'longitude', 'avg_price_1km']].to_csv(output_path, index=False)

    latitude   longitude  avg_price_1km
0  40.696524  -73.991617       4.990149
1  40.766115  -73.989040       5.154065
2  40.808110  -73.943756       4.700019
3  37.772004 -122.431619       5.193322
4  38.925627  -77.034596       4.944364
5  37.753164 -122.429526       5.257195
6  33.980454 -118.462821       5.227422
7  34.046737 -118.260439       4.950736
8  37.781128 -122.501095       5.006894
9  33.992563 -117.895997       4.046230

📈 Correlación de avg_price_1km con log_price:
               avg_price_1km  log_price
avg_price_1km       1.000000   0.531042
log_price           0.531042   1.000000


In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Cargar el dataset limpio
file_path = '/Users/chants/Desktop/Pset2_cucalon/data/processed/Airbnb_Cleaned.csv'
df = pd.read_csv(file_path)

# Copia del DataFrame para no modificar el original
df_room_type = df.copy()

# Verificar que `room_type` existe
if 'room_type' not in df_room_type.columns:
    print("⚠️ La columna 'room_type' no está en el DataFrame. Verifica el dataset.")
else:
    # Aplicar One-Hot Encoding a `room_type`
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    encoded_data = encoder.fit_transform(df_room_type[['room_type']])

    # Convertir el encoding en DataFrame
    encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['room_type']))

    # Resetear índices antes de concatenar
    df_room_type.reset_index(drop=True, inplace=True)
    encoded_df.reset_index(drop=True, inplace=True)

    # Concatenar con el dataset original y eliminar la columna original
    df_room_type = pd.concat([df_room_type.drop(columns=['room_type']), encoded_df], axis=1)

    # Verificar que todas las columnas sean numéricas
    df_room_type = df_room_type.select_dtypes(include=['number'])

    # Calcular la correlación con `log_price`
    correlation_matrix = df_room_type.corr()
    correlation_log_price = correlation_matrix[['log_price']].sort_values(by='log_price', ascending=False)

    # Mostrar la correlación
    print("\n📈 Correlación de room_type con log_price:")
    print(correlation_log_price)

    # Guardar las columnas codificadas en un archivo CSV
output_path = "/Users/chants/Desktop/Pset2_cucalon/data/processed/room_type_feature.csv"
df_room_type.to_csv(output_path, index=False)




📈 Correlación de room_type con log_price:
                           log_price
log_price                   1.000000
room_type_Entire home/apt   0.602500
accommodates                0.567574
bedrooms                    0.536171
beds                        0.441953
latitude                   -0.002193
longitude                  -0.047529
review_scores_rating       -0.070754
room_type_Shared room      -0.222487
room_type_Private room     -0.531648


In [44]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# 📌 Cargar el dataset limpio
file_path = '/Users/chants/Desktop/Pset2_cucalon/data/processed/Airbnb_Cleaned.csv'
df = pd.read_csv(file_path)

# 📌 Copia del DataFrame para no modificar el original
df_bed_type = df.copy()

# 📌 Aplicar One-Hot Encoding a `bed_type`
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')  # No eliminamos categorías para evitar pérdida de info
encoded_data = encoder.fit_transform(df_bed_type[['bed_type']])

# 📌 Convertir el encoding en DataFrame con nombres de columnas correctos
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['bed_type']))

# 📌 Resetear índices antes de concatenar
df_bed_type.reset_index(drop=True, inplace=True)
encoded_df.reset_index(drop=True, inplace=True)

# 📌 Concatenar y eliminar la columna original
df_bed_type = pd.concat([df_bed_type.drop(columns=['bed_type']), encoded_df], axis=1)

# 📌 Filtrar solo columnas numéricas para calcular la correlación
df_numeric = df_bed_type.select_dtypes(include=['number'])

# 📌 Calcular la correlación con `log_price`
correlation_matrix = df_numeric.corr()
correlation_log_price = correlation_matrix[['log_price']].sort_values(by='log_price', ascending=False)

# 📌 Mostrar la correlación
print("\n📈 Correlación de bed_type con log_price:")
print(correlation_log_price)

# 📌 Guardar las columnas codificadas en un archivo CSV
output_path = "/Users/chants/Desktop/Pset2_cucalon/data/processed/bed_type_feature.csv"
df_bed_type.to_csv(output_path, index=False)



📈 Correlación de bed_type con log_price:
                        log_price
log_price                1.000000
accommodates             0.567574
bedrooms                 0.536171
beds                     0.441953
bed_type_Real Bed        0.099230
latitude                -0.002193
bed_type_Pull-out Sofa  -0.039481
bed_type_Couch          -0.039985
longitude               -0.047529
bed_type_Airbed         -0.050427
bed_type_Futon          -0.064557
review_scores_rating    -0.070754


In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Cargar el dataset limpio
file_path = '/Users/chants/Desktop/Pset2_cucalon/data/processed/Airbnb_Cleaned.csv'
df = pd.read_csv(file_path)

# Copia del DataFrame para no modificar el original
df_city = df.copy()

# Aplicar One-Hot Encoding a `city`
encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')  # Elimina la primera categoría para evitar colinealidad
encoded_data = encoder.fit_transform(df_city[['city']])

# Convertir el encoding en DataFrame
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['city']))

# Resetear los índices antes de concatenar
df_city.reset_index(drop=True, inplace=True)
encoded_df.reset_index(drop=True, inplace=True)

# Concatenar con el dataset original y eliminar la columna original
df_city = pd.concat([df_city.drop(columns=['city']), encoded_df], axis=1)

# Filtrar solo columnas numéricas para la correlación
df_city_numeric = df_city.select_dtypes(include=['number'])

# Calcular la correlación con `log_price`
correlation_matrix = df_city_numeric.corr()
correlation_log_price = correlation_matrix[['log_price']].sort_values(by='log_price', ascending=False)

# Mostrar la correlación
print("\n📈 Correlación de city con log_price:")
print(correlation_log_price)

# Guardar las columnas codificadas en un archivo CSV
output_path = "/Users/chants/Desktop/Pset2_cucalon/data/processed/city_feature.csv"
df_city.to_csv(output_path, index=False)



📈 Correlación de city con log_price:
                      log_price
log_price              1.000000
accommodates           0.567574
bedrooms               0.536171
beds                   0.441953
city_SF                0.166738
city_DC                0.082282
latitude              -0.002193
longitude             -0.047529
city_Chicago          -0.051902
city_LA               -0.056585
review_scores_rating  -0.070754
city_NYC              -0.076958


In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Cargar el dataset limpio
file_path = '/Users/chants/Desktop/Pset2_cucalon/data/processed/Airbnb_Cleaned.csv'
df = pd.read_csv(file_path)

# Copia del DataFrame para no modificar el original
df_property_type = df.copy()

# Aplicar One-Hot Encoding a `property_type`
encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')  
encoded_data = encoder.fit_transform(df_property_type[['property_type']])

# Convertir el encoding en DataFrame
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['property_type']))

# Resetear índices antes de concatenar
df_property_type.reset_index(drop=True, inplace=True)
encoded_df.reset_index(drop=True, inplace=True)

# Concatenar y eliminar la columna original
df_property_type = pd.concat([df_property_type.drop(columns=['property_type']), encoded_df], axis=1)

# Filtrar solo columnas numéricas para calcular la correlación
df_numeric = df_property_type.select_dtypes(include=['number'])

# Calcular la correlación con `log_price`
correlation_matrix = df_numeric.corr()
correlation_log_price = correlation_matrix[['log_price']].sort_values(by='log_price', ascending=False)

# Mostrar la correlación
print("\n📈 Correlación de property_type con log_price:")
print(correlation_log_price)

# Guardar las columnas codificadas en un archivo CSV
output_path = "/Users/chants/Desktop/Pset2_cucalon/data/processed/property_type_feature.csv"
df_property_type.to_csv(output_path, index=False)


📈 Correlación de property_type con log_price:
                                  log_price
log_price                          1.000000
accommodates                       0.567574
bedrooms                           0.536171
beds                               0.441953
property_type_Condominium          0.065726
property_type_Loft                 0.041135
property_type_Timeshare            0.036887
property_type_Villa                0.022659
property_type_Other                0.020812
property_type_Boat                 0.017296
property_type_Boutique hotel       0.012821
property_type_House                0.011228
property_type_Townhouse            0.010967
property_type_Vacation home        0.010851
property_type_Castle               0.010826
property_type_Serviced apartment   0.008575
property_type_Earth House          0.005155
property_type_Tipi                 0.004116
property_type_Lighthouse           0.003786
property_type_Train                0.003058
property_type_Yurt           

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Cargar el dataset limpio
file_path = '/Users/chants/Desktop/Pset2_cucalon/data/processed/Airbnb_Cleaned.csv'
df = pd.read_csv(file_path)

# Copia del DataFrame para no modificar el original
df_cleaning_fee = df.copy()

# Aplicar One-Hot Encoding a `cleaning_fee`
encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')  # drop='first' evita colinealidad
encoded_data = encoder.fit_transform(df_cleaning_fee[['cleaning_fee']])

# Convertir el encoding en DataFrame con nombres de columnas correctos
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['cleaning_fee']))

# Resetear índices antes de concatenar
df_cleaning_fee.reset_index(drop=True, inplace=True)
encoded_df.reset_index(drop=True, inplace=True)

# Concatenar y eliminar la columna original
df_cleaning_fee = pd.concat([df_cleaning_fee.drop(columns=['cleaning_fee']), encoded_df], axis=1)

# Filtrar solo columnas numéricas para calcular la correlación
df_numeric = df_cleaning_fee.select_dtypes(include=['number'])

# Calcular la correlación con `log_price`
correlation_matrix = df_numeric.corr()
correlation_log_price = correlation_matrix[['log_price']].sort_values(by='log_price', ascending=False)

# Mostrar la correlación
print("\n📈 Correlación de cleaning_fee con log_price:")
print(correlation_log_price)

# Guardar las columnas codificadas en un archivo CSV
output_path = "/Users/chants/Desktop/Pset2_cucalon/data/processed/cleaning_fee_feature.csv"
df_cleaning_fee.to_csv(output_path, index=False)



📈 Correlación de cleaning_fee con log_price:
                      log_price
log_price              1.000000
accommodates           0.567574
bedrooms               0.536171
beds                   0.441953
cleaning_fee_True      0.111191
latitude              -0.002193
longitude             -0.047529
review_scores_rating  -0.070754


In [ ]:
import pandas as pd

# Cargar el dataset original
file_path = "/Users/chants/Desktop/Pset2_cucalon/data/processed/Airbnb_Cleaned.csv"
df = pd.read_csv(file_path)

# Cargar los archivos de features generados previamente
file_amenities = "/Users/chants/Desktop/Pset2_cucalon/data/processed/amenities_feature.csv"
file_ubi = "/Users/chants/Desktop/Pset2_cucalon/data/processed/ubi_feature.csv"
file_room = "/Users/chants/Desktop/Pset2_cucalon/data/processed/room_type_feature.csv"
file_bed = "/Users/chants/Desktop/Pset2_cucalon/data/processed/bed_type_feature.csv"
file_city = "/Users/chants/Desktop/Pset2_cucalon/data/processed/city_feature.csv"
file_property = "/Users/chants/Desktop/Pset2_cucalon/data/processed/property_type_feature.csv"
file_cleaning = "/Users/chants/Desktop/Pset2_cucalon/data/processed/cleaning_fee_feature.csv"

# Cargar los datasets de features
df_amenities = pd.read_csv(file_amenities)
df_ubi = pd.read_csv(file_ubi)
df_room = pd.read_csv(file_room)
df_bed = pd.read_csv(file_bed)
df_city = pd.read_csv(file_city)
df_property = pd.read_csv(file_property)
df_cleaning = pd.read_csv(file_cleaning)

# Unir las features al dataset original evitando duplicados
df_featured = df.copy()

df_featured = df_featured.merge(df_amenities, left_index=True, right_index=True, how="left", suffixes=("", "_amen"))
df_featured = df_featured.merge(df_ubi, left_index=True, right_index=True, how="left", suffixes=("", "_ubi"))
df_featured = df_featured.merge(df_room, left_index=True, right_index=True, how="left", suffixes=("", "_room"))
df_featured = df_featured.merge(df_bed, left_index=True, right_index=True, how="left", suffixes=("", "_bed"))
df_featured = df_featured.merge(df_city, left_index=True, right_index=True, how="left", suffixes=("", "_city"))
df_featured = df_featured.merge(df_property, left_index=True, right_index=True, how="left", suffixes=("", "_prop"))
df_featured = df_featured.merge(df_cleaning, left_index=True, right_index=True, how="left", suffixes=("", "_clean"))

# Eliminar columnas duplicadas que se generaron en el merge
df_featured = df_featured.loc[:, ~df_featured.columns.duplicated()]

# Verificar si hay valores NaN y llenarlos con 0
df_featured.fillna(0, inplace=True)

# Seleccionar las mejores features
selected_features = [
    "log_price", "avg_price_1km", "accommodates", "bedrooms", "beds",
    "total_amenities", "city_SF", "city_DC", "property_type_Condominium",
    "property_type_Loft", "cleaning_fee_True"
]

# Filtrar solo las columnas necesarias
df_selected = df_featured[selected_features]

# Guardar el nuevo dataset con las features seleccionadas
output_path = "/Users/chants/Desktop/Pset2_cucalon/data/processed/Airbnb_Featured.csv"
df_selected.to_csv(output_path, index=False)

print(f"✅ Archivo Airbnb_Featured.csv creado")

✅ Archivo Airbnb_Featured.csv creado


Escala

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Cargar el dataset original
file_path = "/Users/chants/Desktop/Pset2_cucalon/data/processed/Airbnb_Cleaned.csv"
df = pd.read_csv(file_path)

# Cargar los archivos de features generados previamente
file_amenities = "/Users/chants/Desktop/Pset2_cucalon/data/processed/amenities_feature.csv"
file_ubi = "/Users/chants/Desktop/Pset2_cucalon/data/processed/ubi_feature.csv"
file_room = "/Users/chants/Desktop/Pset2_cucalon/data/processed/room_type_feature.csv"
file_bed = "/Users/chants/Desktop/Pset2_cucalon/data/processed/bed_type_feature.csv"
file_city = "/Users/chants/Desktop/Pset2_cucalon/data/processed/city_feature.csv"
file_property = "/Users/chants/Desktop/Pset2_cucalon/data/processed/property_type_feature.csv"
file_cleaning = "/Users/chants/Desktop/Pset2_cucalon/data/processed/cleaning_fee_feature.csv"

# Cargar los datasets de features
df_amenities = pd.read_csv(file_amenities)
df_ubi = pd.read_csv(file_ubi)
df_room = pd.read_csv(file_room)
df_bed = pd.read_csv(file_bed)
df_city = pd.read_csv(file_city)
df_property = pd.read_csv(file_property)
df_cleaning = pd.read_csv(file_cleaning)

# Unir las features al dataset original evitando duplicados
df_featured = df.copy()
df_featured = df_featured.merge(df_amenities, left_index=True, right_index=True, how="left", suffixes=("", "_amen"))
df_featured = df_featured.merge(df_ubi, left_index=True, right_index=True, how="left", suffixes=("", "_ubi"))
df_featured = df_featured.merge(df_room, left_index=True, right_index=True, how="left", suffixes=("", "_room"))
df_featured = df_featured.merge(df_bed, left_index=True, right_index=True, how="left", suffixes=("", "_bed"))
df_featured = df_featured.merge(df_city, left_index=True, right_index=True, how="left", suffixes=("", "_city"))
df_featured = df_featured.merge(df_property, left_index=True, right_index=True, how="left", suffixes=("", "_prop"))
df_featured = df_featured.merge(df_cleaning, left_index=True, right_index=True, how="left", suffixes=("", "_clean"))

# Eliminar columnas duplicadas que se generaron en el merge
df_featured = df_featured.loc[:, ~df_featured.columns.duplicated()]

# Verificar si hay valores NaN y llenarlos con 0
df_featured.fillna(0, inplace=True)

# Seleccionar las mejores features
selected_features = [
    "log_price", "avg_price_1km", "accommodates", "bedrooms", "beds",
    "total_amenities", "city_SF", "city_DC", "property_type_Condominium",
    "property_type_Loft", "cleaning_fee_True"
]

# Filtrar solo las columnas necesarias
df_selected = df_featured[selected_features]

# Separar `log_price` (objetivo) del resto de variables
X = df_selected.drop(columns=["log_price"])
y = df_selected["log_price"]

# Aplicar StandardScaler solo a las variables numéricas
num_features = ["avg_price_1km", "accommodates", "bedrooms", "beds", "total_amenities"]
scaler = StandardScaler()
X[num_features] = scaler.fit_transform(X[num_features])

# Volver a unir `log_price` con las features escaladas
df_final = X.copy()
df_final["log_price"] = y  # Mantener log_price sin escalar

# Guardar el dataset final con las features escaladas
output_path = "/Users/chants/Desktop/Pset2_cucalon/data/processed/Airbnb_Featured_Scaled.csv"
df_final.to_csv(output_path, index=False)

print(f"✅ Archivo Airbnb_Featured_Scaled.csv creado con éxito")


✅ Archivo Airbnb_Featured_Scaled.csv creado con éxito
